# Masterclass de Hiperparámetros en XGBoost

Configurar un modelo de Boosting es como afinar un instrumento musical de alta precisión. Cada parámetro controla un aspecto del equilibrio entre **aprender lo suficiente** (evitar Underfitting) y **no memorizar demasiado** (evitar Overfitting).

---

## 1. Parámetros de Estructura y Aprendizaje

### `n_estimators`
* **Concepto:** Es el número de árboles que se construirán secuencialmente. 
* **Lógica:** Cada árbol intenta corregir los errores del anterior. 
* **Efecto:** Muy pocos árboles resultan en un modelo débil. Demasiados árboles pueden causar overfitting.

### `learning_rate` (o $\eta$)
* **Concepto:** Es el "tamaño del paso" que da el modelo en cada iteración.
* **Lógica:** Multiplica la contribución de cada nuevo árbol. Si es $0.1$, el nuevo árbol solo aporta el 10% de su "opinión" al modelo final.
* **Efecto:** Un valor bajo hace que el modelo sea más robusto y lento para aprender, lo cual suele mejorar la precisión final (pero requiere más `n_estimators`).

### `max_depth`
* **Concepto:** La profundidad máxima de cada árbol.
* **Lógica:** Controla qué tan complejas son las interacciones que el modelo puede captar.
* **Efecto:** Un valor alto (ej. 10) permite captar relaciones muy finas, pero aumenta drásticamente el riesgo de overfitting. Valores comunes: 3 a 7.

---

## 2. Parámetros de Control de Complejidad (Regularización)

### `min_child_weight`
* **Concepto:** La suma mínima de "instancias" (Hessianos) necesarias en un nodo hijo.
* **Lógica Matemática:** Si el peso en un nodo es menor a este valor, el proceso de partición se detiene.
* **Efecto:** Es una de las herramientas más potentes contra el ruido. Un valor alto evita que el modelo cree ramas para casos muy específicos o aislados.

### `gamma` ($\gamma$)
* **Concepto:** El "peaje" de reducción de pérdida mínima para realizar un split.
* **Lógica Matemática:** Como vimos en la fórmula de ganancia, un nodo solo se divide si la ganancia es mayor a $\gamma$.
* **Efecto:** Hace que el algoritmo sea "conservador". Cuanto más alto sea Gamma, más sencillo (y generalista) será el árbol.

## 3. Parámetros de Muestreo (Estocásticos)

### `subsample`
* **Concepto:** Porcentaje de filas (datos) que se usarán para entrenar cada árbol.
* **Lógica:** Si es $0.8$, cada árbol se entrena con un 80% del dataset elegido al azar.
* **Efecto:** Ayuda a prevenir el overfitting al introducir aleatoriedad (evita que los árboles vean siempre lo mismo).

### `colsample_bytree`
* **Concepto:** Porcentaje de columnas (variables) que se usarán para construir cada árbol.
* **Lógica:** Selecciona un subconjunto de características al azar para cada iteración.
* **Efecto:** Evita que una sola variable dominante eclipse a las demás, obligando al modelo a buscar patrones en otras variables.

### `colsample_bynode`
* **Concepto:** Porcentaje de columnas que se consideran en **cada split** (cada vez que una rama se divide).
* **Lógica:** Es un nivel de aleatoriedad aún más profundo que `bytree`. 
* **Efecto:** Es muy útil cuando tienes cientos de variables correlacionadas entre sí.

## 4. El Parámetro Especial: `scale_pos_weight`

Aquí hay una aclaración importante: **¿Es una técnica de balanceo o un parámetro? ¡Es ambas!**

* **Por qué es un parámetro:** Porque vive dentro de la configuración del objeto `XGBClassifier`.
* **Por qué es una técnica de balanceo:** Porque implementa lo que llamamos **Aprendizaje sensible al costo (Cost-sensitive learning)**. 

### ¿Cómo funciona?
Internamente, XGBoost multiplica el gradiente ($g_i$) y el hessiano ($h_i$) de los casos positivos por este valor. 
* Si `scale_pos_weight = 10`, el algoritmo siente que fallar en un caso positivo es **10 veces más doloroso** que fallar en uno negativo.
* **Cálculo ideal:** $\frac{\text{Suma de ejemplos negativos}}{\text{Suma de ejemplos positivos}}$.

## Resumen Didáctico: El Equipo de Construcción

Para cerrar nuestro notebook, imagina que XGBoost es una **obra de construcción**:

1. **`n_estimators`**: Es el número de obreros que trabajan en fila.
2. **`learning_rate`**: Es qué tan rápido les permites trabajar (si van muy rápido, cometen errores).
3. **`max_depth`**: Es cuántos pisos puede tener el andamio de cada obrero.
4. **`min_child_weight`**: Es el mínimo de materiales necesarios para que un obrero empiece una pared (si hay poco, no vale la pena).
5. **`gamma`**: Es el inspector de calidad; si el cambio no es lo suficientemente bueno, no deja construir.
6. **`subsample` y `colsample`**: Son las herramientas y planos que les das al azar para que no todos construyan exactamente igual.
7. **`scale_pos_weight`**: Es una alarma especial que suena 10 veces más fuerte si un obrero comete un error en los acabados de lujo (clase minoritaria).

# Los Especialistas Faltantes: Regularización y Optimización

Para que nuestro "Equipo de Construcción" no solo termine la obra, sino que sea resistente y eficiente, necesitamos añadir estos parámetros:

---

## 5. El Dúo de la Regularización (L1 y L2)

XGBoost brilla por su capacidad de evitar el overfitting gracias a estos dos pesos matemáticos que se añaden a la Función Objetivo:

### `reg_alpha` (Regularización L1 / Lasso)
* **Concepto:** Penaliza la suma de los valores absolutos de los pesos de las hojas.
* **Lógica Matemática:** $\alpha \sum |w_j|$.
* **Efecto:** Ayuda a la **selección de variables**. Tiende a hacer que los pesos de las variables menos importantes sean exactamente cero. 
* **Analogía:** Es un "limpiador" que quita del plano las herramientas que no se están usando para que el obrero no se distraiga.

### `reg_lambda` (Regularización L2 / Ridge)
* **Concepto:** Penaliza la suma de los cuadrados de los pesos de las hojas.
* **Lógica Matemática:** $\lambda \sum w_j^2$.
* **Efecto:** Suaviza los pesos. Evita que una sola variable tome un control exagerado del modelo.
* **Analogía:** Es un "estabilizador" que asegura que ninguna columna del edificio soporte todo el peso ella sola, repartiendo la carga.

---

## 6. Parámetros de Control y Hardware

### `random_state` (o `seed`)
* **Concepto:** La semilla para la generación de números aleatorios.
* **Uso:** Fundamental para la **reproducibilidad**. 
* **Efecto:** Asegura que si corres el modelo hoy y mañana, obtengas exactamente los mismos resultados.

### `n_jobs`
* **Concepto:** Número de hilos de procesamiento (cores de CPU) a utilizar.
* **Efecto:** Si pones `-1`, XGBoost usará toda la potencia de tu computadora para entrenar lo más rápido posible. Es pura velocidad de ejecución.

### `tree_method`
* **Concepto:** El algoritmo de construcción de árboles (ej: `auto`, `exact`, `approx`, `hist`).
* **Uso:** En datasets muy grandes (millones de filas), se usa `hist` (histograma) para que el entrenamiento sea 10 veces más rápido sin perder mucha precisión.

## 7. El Toque Final: `early_stopping_rounds`

Este no es un hiperparámetro estático, sino una **estrategia de entrenamiento**.

* **Concepto:** Detiene el entrenamiento si el modelo deja de mejorar en un set de validación externo.
* **Efecto:** Evita desperdiciar tiempo y previene el overfitting. Si pones `early_stopping_rounds=10`, y tras 10 árboles nuevos el error no baja, el modelo dice: "Ya terminé, no voy a mejorar más".

## Resumen Didáctico Actualizado (El Final de la Obra)

Para completar nuestra analogía de la **obra de construcción**, añadimos:

8. **`reg_alpha` (L1)**: El minimalista. Si una decoración no es esencial, la borra del plano para no perder tiempo.
9. **`reg_lambda` (L2)**: El ingeniero estructural. Se asegura de que el peso del edificio esté bien distribuido entre todas las vigas.
10. **`random_state`**: El diario de obra. Permite repetir la construcción paso a paso exactamente igual en otro terreno.
11. **`tree_method='hist'`**: El uso de prefabricados. En lugar de medir cada ladrillo, usa bloques estándar para ir mucho más rápido.
12. **`early_stopping_rounds`**: El contrato de eficiencia. Si el edificio ya no se ve mejor por más que trabajemos, el cliente deja de pagar y cerramos la obra.